In [20]:
import os
import time
import numpy as np
import pickle
import matplotlib.pyplot as plt
import loader
import torch
from torch import optim
from torch import nn
from config import config_base
from config import config_r_net
from config import config_match_lstm
from config import config_bi_daf
import preprocess_data
import utils
from modules.layers.loss import MyNLLLoss
from modules.layers.loss import RougeLoss
from modules import match_lstm
from modules import r_net
from modules import bi_daf


# config = config_match_lstm.config
# config = config_r_net.config
config = config_bi_daf.config

In [21]:
lang = loader.load_vocab(config.vocab_path)

In [22]:
embedding_np = loader.load_w2v(config.embedding_path)

In [23]:
    if os.path.isfile(config.train_pkl):
        with open(config.train_pkl, 'rb') as file:
            train_data = pickle.load(file)
    else:
        train_data = loader.load_data(config.train_df, lang)
        with open(config.train_pkl, 'wb') as file:
            pickle.dump(train_data, file)

    # load val data
    if os.path.isfile(config.val_pkl):
        with open(config.val_pkl, 'rb') as file:
            val_data = pickle.load(file)
    else:
        val_data = loader.load_data(config.val_df, lang)
        with open(config.val_pkl, 'wb') as file:
            pickle.dump(val_data, file)

In [24]:
    train_loader = loader.build_loader(
        dataset=train_data,
        batch_size=config.batch_size,
        shuffle=True,
        drop_last=True
    )

In [26]:
    param = {
        'embedding': embedding_np,
        'embedding_type': config.embedding_type,
        'embedding_is_training': config.embedding_is_training,
        'mode': config.mode,
        'hidden_size': config.hidden_size,
        'dropout_p': config.dropout_p,
        'encoder_dropout_p': config.encoder_dropout_p,
        'encoder_bidirectional': config.encoder_bidirectional,
        'encoder_layer_num': config.encoder_layer_num,
        'is_bn': config.is_bn
    }
    model = eval(config.model_name).Model(param)
    model = model.cuda()

In [27]:
    if config.criterion == 'RougeLoss':
        criterion = eval(config.criterion)(lam=config.lamda)
    else:
        criterion = eval(config.criterion)()

In [28]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [29]:
    if config.is_for_rouge:
        model_path = os.path.join('model', config.model_save+'_mrt')
        if os.path.isfile(model_path):
            print('load training param, ', model_path)
            state = torch.load(model_path)
            model.load_state_dict(state['cur_model_state'])
            optimizer.load_state_dict(state['cur_opt_state'])
            train_loss_list = state['train_loss']
            val_loss_list = state['val_loss']
            steps = state['steps']
            time_use = state['time']

        else:
            model_path = os.path.join('model', config.model_save)
            assert os.path.isfile(model_path)
            state = torch.load(model_path)
            model.load_state_dict(state['best_model_state'])
            optimizer.load_state_dict(state['best_opt_state'])
            train_loss_list = []
            val_loss_list = []
            steps = []
            time_use = 0
        epoch_list = range(1)

    else:
        model_path = os.path.join('model', config.model_save)
        if os.path.isfile(model_path):
            print('load training param, ', model_path)
            state = torch.load(model_path)
            model.load_state_dict(state['cur_model_state'])
            optimizer.load_state_dict(state['cur_opt_state'])
            epoch_list = range(state['cur_epoch']+1, state['cur_epoch']+1+config.epoch)
            train_loss_list = state['train_loss']
            val_loss_list = state['val_loss']
            steps = state['steps']
            time_use = state['time']
        else:
            state = None
            epoch_list = range(config.epoch)
            train_loss_list = []
            val_loss_list = []
            steps = []
            time_use = 0

load training param,  model/bi_daf_1


In [30]:
batch0 = 0
for batch in train_loader:
    batch0 = batch
    break

In [33]:
batch = utils.deal_batch(batch0)

In [27]:
torch.log(torch.tensor(0.1))

tensor(-2.3026)

In [36]:
outputs.size()

torch.Size([2, 32, 285])

In [37]:
loss_value = criterion(outputs, batch)

In [38]:
loss_value

tensor(1.4996, device='cuda:0')

In [2]:
import torch

In [3]:
a = torch.Tensor([1,2])

In [12]:
a.type() == torch.FloatTensor

False

In [14]:
a.type()

'torch.FloatTensor'